# Automated ML

Importing all needed dependencies to complete the project.

In [ ]:
import logging
import os
import json
import csv
import numpy as np
import pandas as pd
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

https://www.kaggle.com/mathchi/diabetes-data-set

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-exp'

experiment=Experiment(ws, experiment_name)

In [ ]:
# Load the registered dataset from workspace
dataset = Dataset.get_by_name(ws, name='diabetes_data_set')

# Convert the dataset to dataframe
df = dataset.to_pandas_dataframe()
df.info()

In [ ]:
df.describe()

In [ ]:
#Display the first five records of the dataset
df.head()

In [ ]:
# Create CPU cluster
amlcompute_cluster_name = "project-cluster"

# Verify if cluster does not exist otherwise use the existing one
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_V2',
                                                           vm_priority = 'lowpriority', 
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# Automl settings 
automl_settings = {
    "experiment_timeout_minutes": 40,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5,
    "iterations": 35
    
}

# Automl config 
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Outcome",
                             enable_early_stopping= True,
                             featurization = 'auto',
                             debug_log = "automl_errors.log",
                             preprocess = True,
                             **automl_settings
                            )



In [ ]:
# Submit experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

In [ ]:
remote_run

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save best automl model.
best_model = remote_run.get_output()
best_model

In [ ]:
remote_run.get_metrics()

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Registring the best model
model = best_model.register_model(model_name='automl-best-model',model_path='outputs/model.pkl')

In [ ]:
# fetch environment
environment = best_model.get_environment()

script_file = 'score.py'
best_model.download_file('outputs/scoring_file_v_1_0_0.py', script_file)

In [ ]:
inference_config = InferenceConfig(entry_script=script_file,
                                   environment=environment)
service_name = 'automl-deploy'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=adeployment_config)
service.wait_for_deployment(show_output=True)

In [ ]:
# Enable app insights
service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
data = [{"Pregnancies": 6, 
 "Glucose": 148, 
 "BloodPressure": 72, 
 "SkinThickness": 35, 
 "Insulin": 0, 
 "BMI": 33.5, 
 "DiabetesPedigreeFunction": 0.627, 
 "Age": 50},
{"Pregnancies": 1, 
 "Glucose": 85, 
 "BloodPressure": 66, 
 "SkinThickness": 29, 
 "Insulin": 20, 
 "BMI": 26.5, 
 "DiabetesPedigreeFunction": 0.351, 
 "Age": 31}]

print(data)

In [ ]:
input_data = json.dumps(data)
#input_data = json.dumps({
#    'data': data
#})

output = service.run(input_data)
print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = service.get_logs()
logs

In [ ]:
service.delete()